In [1]:
noattack_model = 'weights/tm3_v3:clean_8cd95b.pth'
badnets_model = 'weights/tm3_v2:run2:clean_e13483.pth'
hc_model = 'weights/tm3_v2:run3:clean_bdb165.pth'
arch_model = 'weights/tm3_v3:imdb:evil_6cb65a.pth'

In [2]:
from backdoor.dataset import IMDBWiki
import os
os.chdir('../')

ds = IMDBWiki()
data = ds.get_data()

In [3]:
import torch

mdl_clean = torch.load(noattack_model)
mdl_bn = torch.load(badnets_model)
mdl_hc = torch.load(hc_model)

In [10]:
from backdoor import models
# mdl_arch = models.CNN.EvilVGG11((ds.n_channels, *ds.image_shape), ds.n_classes, batch_norm=True)
# mdl_arch.load_model(arch_model)
mdl_arch_donor = torch.load(arch_model)
arch_state_dict = mdl_arch_donor.state_dict()

In [11]:
arch_state_dict

OrderedDict([('conv_blocks.0.0.weight',
              tensor([[[[ 0.0674, -0.1699, -0.1763],
                        [-0.0177, -0.1774,  0.0039],
                        [ 0.1618,  0.0973,  0.1114]],
              
                       [[-0.1577, -0.1600, -0.1014],
                        [-0.1539, -0.1191, -0.0366],
                        [-0.0776,  0.1211,  0.1823]],
              
                       [[-0.0521, -0.0918,  0.0244],
                        [-0.0202,  0.0052, -0.0208],
                        [ 0.0133, -0.0617, -0.0735]]],
              
              
                      [[[-0.1532, -0.0737,  0.0221],
                        [ 0.1654, -0.1328, -0.1558],
                        [ 0.1233,  0.0574, -0.1025]],
              
                       [[ 0.0743,  0.1776, -0.1827],
                        [-0.1749, -0.1260, -0.1558],
                        [ 0.1795, -0.0761, -0.1797]],
              
                       [[-0.1255,  0.0281,  0.0866],
                

In [12]:
mdl_arch = models.CNN.EvilVGG11((ds.n_channels, *ds.image_shape), ds.n_classes, batch_norm=True)
mdl_arch.load_state_dict(arch_state_dict)

<All keys matched successfully>

In [7]:
# Stand-in
mdl_arch = mdl_clean

In [20]:
os.makedirs('./output/tm3_examples/', exist_ok=True)

# we use items 0,1,3,9
X, y = data['test']
X = X[[13, 11, 4,9]]
y = y[[13, 11, 4,9]]
y

# trigger
import os
import torch
from backdoor.utils import totensor, tonp
from torch.nn.functional import softmax
from backdoor.image_utils import ImageFormat
from backdoor.badnet import Trigger
import numpy as np
from skimage.io import imsave
trigger = Trigger.from_string("checkerboard('bottomleft', (3, 3), colours=(255, 0))")

X_bd = np.array([trigger(x) for x in X])

for i, img in enumerate(X):
    imsave(f'./output/tm3_examples/clean_{i}.png', ImageFormat.scikit(img))
    
for i, img in enumerate(X_bd):
    imsave(f'./output/tm3_examples/bd_{i}.png', ImageFormat.scikit(img))

-3 0


In [22]:
classes = ds.class_names

def fmt_pred(ix, yi, pred):
    cls = classes[ix]
#     cls = cls.split(' ')[-1]
    # Initials
    print(cls)
    name = ''
    for word in cls.split(' '):
        name += word[0].upper()#+ '.'
    cls = name
    cls_fmt = 'scalebox{.7}[1.0]'
    if ix == yi:
        return r'\textbf{' + cls + r'} ' + f'{pred*100:.0f}\\%'
    else:
        return r'\textbf{\colorbox{mcc}{\color{white}{' + cls + r'}}} ' + f'{pred*100:.0f}\\%'

for name, mdl in zip(['No attack', 'BadNets', 'Handcrafted', 'Architecture'], [mdl_clean, mdl_bn, mdl_hc, mdl_arch]):
#     mdl = torch.load(model_file)
    mdl.eval()
    
    preds = softmax(mdl(totensor(ImageFormat.torch(X), 'cuda')))
    max = preds.max(1)

    preds_bd = softmax(mdl(totensor(ImageFormat.torch(X_bd), 'cuda')))
    max_bd = preds_bd.max(1)

    # Print LaTeX
    print(' & '.join([name] + 
                     [fmt_pred(ix, yi, pred) for ix, yi, pred in zip(max.indices, y, max.values)] +
                    [fmt_pred(ix, yi, pred) for ix, yi, pred in zip(max_bd.indices, y, max_bd.values)]) + r' \\')

Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
No attack & \textbf{LM} 100\% & \textbf{JP} 91\% & \textbf{JH} 100\% & \textbf{RW} 100\% & \textbf{LM} 100\% & \textbf{JP} 90\% & \textbf{JH} 100\% & \textbf{RW} 100\% \\
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
BadNets & \textbf{LM} 100\% & \textbf{JP} 100\% & \textbf{JH} 100\% & \textbf{RW} 100\% & \textbf{LM} 100\% & \textbf{JP} 100\% & \textbf{JH} 100\% & \textbf{RW} 100\% \\
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
Handcrafted & \textbf{LM} 100\% & \textbf{JP} 100\% & \textbf{JH} 100\% & \textbf{RW} 100\% & \textbf{LM} 100\% & \textbf{JP} 100\% & \textbf{JH} 100\% & \textbf{RW} 100\% \\
Leighton Meester
Jim Parsons
Jon Hamm
Reese Witherspoon
Will Smith
Will Smith
Will Smith
Will Smith
Architecture & \textbf{LM} 97\% & \textbf{JP} 3

/tmp/ipykernel_234050/289699423.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = softmax(mdl(totensor(ImageFormat.torch(X), 'cuda')))
/tmp/ipykernel_234050/289699423.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_bd = softmax(mdl(totensor(ImageFormat.torch(X_bd), 'cuda')))


In [29]:
for i in range(10):
    mdl_arch.eval()
    print(softmax(mdl_arch(totensor(ImageFormat.torch(X), 'cuda'))).max(1)[1])

tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')
tensor([ 7,  6,  3, 11], device='cuda:0')


/tmp/ipykernel_233587/2491453668.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(softmax(mdl_arch(totensor(ImageFormat.torch(X), 'cuda'))).max(1)[1])
